# Eat Safe, Love

## Notebook Set Up

In [ ]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
client = MongoClient('mongodb://localhost:27017/')

db = client['uk_food']

collections = db.list_collection_names()
print(collections)

# Example output
print(["establishments", "another_collection", "another_example"])

# To get the count of documents in a collection
count = db['establishments'].count_documents({})
print(f"Number of establishments: {count}")

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {'HygieneScore': 20}

# Use count_documents to display the number of documents in the result
count = db['establishments'].count_documents(query)
print(f"Number of establishments with a hygiene score of 20: {count}")

# Display the first document in the results using pprint
first_document = db['establishments'].find_one(query)
print("First document with hygiene score of 20:")
pprint(first_document)

In [ ]:
# Convert the result to a Pandas DataFrame
documents = list(db['establishments'].find(query))  # fetch all matching documents
df = pd.DataFrame(documents)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {df.shape[0]}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {
    'LocalAuthorityName': {'$regex': 'London', '$options': 'i'},  # case-insensitive regex match
    'RatingValue': {'$gte': 4}
}

# Use count_documents to display the number of documents in the result
count = db['establishments'].count_documents(query)
print(f"Number of establishments in London with a RatingValue >= 4: {count}")

# Get all documents that match the query
documents = list(db['establishments'].find(query))
# Display the first document in the results using pprint
pprint(documents[0]) if documents else print("No results found.")

In [ ]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(documents)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {df.shape[0]}")

# Display the first 10 rows of the DataFrame
print("\nFirst 10 rows of the DataFrame:")
print(df.head(10))

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

latitude = 51.4768  
longitude = -0.0005

degree_search = 0.01

query = {
    'RatingValue': 5,
    'Latitude': {'$gte': latitude - degree_search, '$lte': latitude + degree_search},
    'Longitude': {'$gte': longitude - degree_search, '$lte': longitude + degree_search}
}
  
sort = [('HygieneScore', 1)]

results = list(db['establishments'].find(query).sort(sort).limit(5))

# Print the results
pprint(results)

In [ ]:
# Convert result to Pandas DataFrame
df = pd.DataFrame(results)
print(f"Number of rows in the DataFrame: {df.shape[0]}")
print(df.head(10))

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline to get the count of establishments with a hygiene score of 0 in each Local Authority
pipeline = [
    {'$match': {'HygieneScore': 0}},
    {'$group': {
        '_id': '$LocalAuthorityName',  # Group by Local Authority Name
        'count': {'$sum': 1}  # Count the number of establishments
    }},
    {'$sort': {'count': -1}}  # Sort the results by count in descending order
]

# Run the aggregation pipeline
results = list(db['establishments'].aggregate(pipeline))

# Print the number of documents in the result
print(f"Number of documents in the result: {len(results)}")

# Print the first 10 results
pprint(results[:10])

In [ ]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))